In [1]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

In [2]:
import threading

In [3]:
threading.active_count()

5

In [4]:
Rsphere = 0.34
Espread = 4.0e-3
npt = 4096*2
nturn = 5

# Original Settings

In [5]:
elems0,lattices,labor = mli.readInputfile('mli.in.t3_iota_8_4_t0p4.optimized')
del(lattices[-1])

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [6]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [7]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)
elems[2].driftexact = 1

In [8]:
labor = ['iotaline','mapout','fin']

In [9]:
mli.writeInputfile(elems,lattices,labor)
mli.run()

### Map

In [10]:
M,G = mli.readTransferMap()
M

,1,2,3,4,5,6
1,0.999968,0.000070,0.000000,0.000000e+00,0.0,-1.147949e-06
2,-1.454445,0.999930,0.000000,0.000000e+00,0.0,-2.587771e-07
3,0.000000,0.000000,0.999995,3.984744e-07,0.0,0.000000e+00
4,0.000000,0.000000,-1.454438,1.000004e+00,0.0,0.000000e+00
5,0.000002,-0.000001,0.000000,0.000000e+00,1.0,-1.297155e+00
6,0.000000,0.000000,0.000000,0.000000e+00,0.0,1.000000e+00


In [11]:
G.head()

,exponents,GP
28,f( 30 00 00 ),7.765582e-12
29,f( 21 00 00 ),-5.701697e-12
33,f( 20 00 01 ),7.272738e-01
34,f( 12 00 00 ),-2.987872e-12
38,f( 11 00 01 ),5.580342e-12


### Sextupoles

In [12]:
indexSext = []
SextStrength = []
for i,item in enumerate(elems):
    if 'elem' in item.keys():
        if 'sextupole' == item.elem:
            indexSext.append(i)
            SextStrength.append(item.g2)
nSext = len(indexSext)

# Optimize

In [13]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx

In [14]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
# G2_ref

In [15]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2
# G3_ref

In [16]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0
# G4_ref

In [17]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [18]:
print(len(G2_ref),len(G3_ref),len(G4_ref),len(G5_ref))

18 37 65 109


In [19]:
elemListTrack = elems[:3]
raysin  =mli.getElem.raytrace(file1='rays.in',type='readonly')
dump    =mli.getElem.particledump(file='rays.out',precision=15)
track   =mli.getElem.autotrack(type='symplectic',order=5)
fin     =mli.getElem.fin
nlinsert = mli.getElem.nlinsert(steps=100)
readmap = mli.getElem.tmi(name='readmap')
savemap = mli.getElem.stm(name='savemap')
getmap    = mli.getElem.gtm(name='getmap')
elemListTrack = elemListTrack + [raysin,dump,track,fin,nlinsert,readmap,savemap,getmap]
trackLine = [mli.getElem.line(name='trackLine',elemList = ['nlinsert','getmap',dump.name])]
trackLabor = mli.buildLabor(['readmap','savemap','clear',raysin.name,track.name,str(nturn)+'*'+trackLine[0].name,'fin'])
mli.writeInputfile(elemListTrack,trackLine,trackLabor)

### Prepare particles

In [20]:
def MLI2norm(data_in,sign=1):
    data=data_in.copy()
    data[:,5] = np.sqrt(1.0-2.0*data[:,5]/bet0+data[:,5]**2)-1.0
    data[:,1] = (data[:,0]*alfx*sign/np.sqrt(betx) + data[:,1]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx*sign/np.sqrt(betx) + data[:,3]/(1+data[:,5])*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    return data
    
def norm2MLI(data_in,sign=1):
    data=data_in.copy()
    data[:,1] = (-data[:,0]*alfx*sign + data[:,1])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,3] = (-data[:,2]*alfx*sign + data[:,3])*NL_c/np.sqrt(betx)*(1+data[:,5])
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c
    data[:,5] = -np.sqrt((1.0+data[:,5])**2+1.0/(bet0*gam0)**2)+1.0/bet0
    return data

In [21]:
def getTBT(npt,nturn,fname='rays.out'):
    TBT = np.loadtxt(fname)
    TBT = TBT[:npt*nturn,:6]
    out = np.zeros([npt,nturn,6])
    for i in range(nturn):
        out[:,i,:] = TBT[i*npt:(i+1)*npt,:]
        out[:,i,:] = MLI2norm(out[:,i,:])
    return out,TBT[npt*(nturn-1):,:]

In [22]:
def getInv(xn,pxn,yn,pyn,delta,tau=NL_t):
    z = xn + 1j*yn
    U = np.real(z/np.sqrt(1-z**2)*np.arcsin(z))
    W = np.real(2*xn/np.sqrt(1-z**2)*np.arcsin(z))
    Hn = 0.5*(xn**2+pxn**2+yn**2+pyn**2)   +tau*U/(1.0+delta)
    In = (xn*pyn -yn*pxn)**2 +xn**2+pxn**2 +tau*W/(1.0+delta)
    return Hn,In

def getInvTBT(TBT):
    npt,nturn,dummy = TBT.shape
    InvTBT = np.zeros([npt,nturn,2])
    for iturn in range(nturn):
        data = TBT[:,iturn,:]
        for ipt in range(npt):
            xn    = data[ipt,0]
            pxn   = data[ipt,1]
            yn    = data[ipt,2]
            pyn   = data[ipt,3]
            delta = data[ipt,5]
            InvTBT[ipt,iturn,:] = getInv(xn,pxn,yn,pyn,delta)
    return InvTBT

In [23]:
def getINVobj(INV,INV0):
    obj=0.0
    tmpHmax = 0.0
    tmpImax = 0.0
    for i in range(nturn):
        tmpH = (INV[:,i,0]/INV0[:,0,0]-1.0)**2 
        tmpI = (INV[:,i,1]/INV0[:,0,1]-1.0)**2
        tmpHmax = np.max([np.max(tmpH),tmpHmax])
        tmpImax = np.max([np.max(tmpI),tmpImax])
        tmp = np.sum(tmpH) + np.sum(tmpI)
        obj = obj + tmp
        if obj > 10:
            return 200*np.sqrt(obj)/(npt*2)/nturn, np.max([tmpHmax,tmpImax])*15
        obj = obj + tmp
    return 200*np.sqrt(obj)/(npt*2)/nturn, np.max([tmpHmax,tmpImax])*15

In [24]:
pData=np.zeros([npt,6])
x = np.random.random([npt,4])
c0 = np.cos(x[:,0])
s0 = np.sin(x[:,0])
c1 = np.cos(x[:,1])
s1 = np.sin(x[:,1])
c2 = np.cos(x[:,2])
s2 = np.sin(x[:,2])
pData[:,0] = Rsphere*c0
pData[:,1] = Rsphere*s0*c1
pData[:,2] = Rsphere*s0*s1*c2
pData[:,3] = Rsphere*s0*s1*s2
pData[:,5] = (x[:,3]-0.5)*2*Espread
pDataIn = norm2MLI(pData,sign=1)

In [25]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexSext):
        elems[j]['g2']=arg[i]
        
    mli.writeInputfile(elems,lattices,labor)
    mli.run()
    M,G = mli.readTransferMap()
    
    obj1 = np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e-10* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2)
    print(obj1)
    if obj1 > 4.0:
        os.chdir('..')
        shutil.rmtree(target)
        return obj1
    else:        
        readmap.map2file(M,G)
        # prepare particles
        np.savetxt('rays.in',pDataIn)
        
        mli.writeInputfile(elemListTrack,trackLine,trackLabor)
        mli.run(4)
        TBT,pDataOut = getTBT(npt,nturn)
        if(np.isnan(TBT.max())):
            os.chdir('..')
            shutil.rmtree(target)
            return obj1#*1.0e6
        Inv0 = getInvTBT(pData.reshape(npt,1,6))
        Inv = getInvTBT(TBT)
#         print(Inv0.shape,Inv.shape,pDataOut.shape)
        obj2, obj3 = getINVobj(Inv,Inv0)
#         print('obj2, obj3',obj2, obj3)
        if obj2 > 1.0 or obj3 > 1.0:
            os.chdir('..')
            shutil.rmtree(target)
            return obj1*obj2*obj3
        
        obj2tot = obj2
        for i in range(200):
            np.savetxt('rays.in',pDataOut)
            if i<4:
                mli.run(8)
            elif i<12:
                mli.run(16)
            elif i<36:
                mli.run(32)
            elif i<120:
                mli.run(32)
            else:
                mli.run(32)
            TBT,pDataOut = getTBT(npt,nturn)
            if(np.isnan(TBT.max())):
                os.chdir('..')
                shutil.rmtree(target)
                return obj1*obj2*obj3
            Inv0 = getInvTBT(pData.reshape(npt,1,6))
            Inv = getInvTBT(TBT)
            dummy2,dummy3 = getINVobj(Inv,Inv0)
#             print('obj2, obj3',dummy2, dummy3)
            obj3 = np.max([obj3,dummy3])
    #             print(Inv0.shape,Inv.shape,pDataOut.shape)
            if dummy2 > 1.0 or np.isnan(dummy2) or obj3 > 1.0:
                os.chdir('..')
                shutil.rmtree(target)
                return obj1*obj2*obj3
            obj2tot = obj2tot + dummy2
            obj2 = (obj2tot/(i+2))*(0.7**(i+1))
        
    os.chdir('..')
    shutil.rmtree(target)
    return obj1*obj2*obj3

In [26]:
print(len(indexSext))
print(len(SextStrength))
print(SextStrength)

18
18
[-288.16075900895, 53.498938900431, 27.418933457304, -166.63098250485, 43.911701881252, 18.283820647369, -148.21072166509, -28.795835464171, 82.814208089224, 91.268894402938, -149.50167972859, -7.7325570968846, -96.562559725056, 97.275312576464, 204.24632300067, -95.569652221046, -741.97015892673, -253.68819753377]


In [27]:
objFunc(SextStrength)

968911.2280080072

In [28]:
#%% run optim
bounds = []
for g2 in SextStrength:
    bounds.append((-200.0,200.0))
print(bounds)    
# result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*4,
#                                         disp=True, polish=True, maxtime=60*10) 
                                        # stop running at maximum 1 min


[(-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0), (-200.0, 200.0)]


In [29]:
# import pickle
# with open('result.thick.DA.sext','wb') as fp:
#     pickle.dump(result,fp)

In [30]:
import pickle
with open('result.thick.DA.sext','rb') as fp:
    result=pickle.load(fp)
result.population_energies[0]

7.398224492838347

In [31]:
result

             message: 'Maximum time has been exceeded.'
                 nit: 0
          population: array([[0.12323141, 0.45850262, 0.62062821, ..., 0.26857225, 0.03280818,
        0.11947396],
       [0.10597849, 0.46023252, 0.61581335, ..., 0.24780488, 0.0012928 ,
        0.16498887],
       [0.13274535, 0.45189085, 0.62558137, ..., 0.27591262, 0.03545463,
        0.09943721],
       ...,
       [0.11250643, 0.46659712, 0.61474531, ..., 0.2461304 , 0.00858528,
        0.17482384],
       [0.12417508, 0.45098647, 0.62318766, ..., 0.24276562, 0.0380288 ,
        0.1697552 ],
       [0.12119441, 0.45098932, 0.62919845, ..., 0.2754812 , 0.01911303,
        0.10396767]])
 population_energies: array([7.39822449, 8.63764198, 8.26729402, 8.50798364, 8.4712174 ,
       8.65802428, 7.96409476, 8.6635107 , 9.26208117, 8.40672191,
       8.83258684, 7.67305179, 8.24998705, 8.97384026, 8.70044514,
       8.34450979, 9.08816675, 8.01280543, 8.95276086, 9.07541396,
       8.65292628, 8.02243355, 

In [32]:
import time

In [ ]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, polish=True, maxtime=60*10)
    with open('result.thick.DA.sext','wb') as fp:
        pickle.dump(result,fp)
    time.sleep(5)

differential_evolution step 0: f(x)= 7.39822
differential_evolution step 0: f(x)= 7.39822
differential_evolution step 1: f(x)= 7.39822
differential_evolution step 2: f(x)= 7.39822
differential_evolution step 3: f(x)= 7.39822
differential_evolution step 4: f(x)= 7.39822
differential_evolution step 5: f(x)= 7.39822
differential_evolution step 6: f(x)= 7.39822
differential_evolution step 7: f(x)= 7.39822
differential_evolution step 8: f(x)= 7.39822
differential_evolution step 9: f(x)= 7.39822
differential_evolution step 10: f(x)= 7.39822
differential_evolution step 11: f(x)= 7.39822
differential_evolution step 12: f(x)= 7.39822
differential_evolution step 13: f(x)= 7.39822
differential_evolution step 14: f(x)= 7.39822
differential_evolution step 15: f(x)= 7.39822
differential_evolution step 16: f(x)= 7.39822
differential_evolution step 17: f(x)= 7.39822
differential_evolution step 18: f(x)= 7.39822
differential_evolution step 19: f(x)= 7.39822
differential_evolution step 20: f(x)= 7.39822

differential_evolution step 3: f(x)= 7.29381
differential_evolution step 4: f(x)= 7.29381
differential_evolution step 5: f(x)= 7.29381
differential_evolution step 6: f(x)= 7.29381
differential_evolution step 7: f(x)= 7.29381
differential_evolution step 8: f(x)= 7.29381
differential_evolution step 9: f(x)= 7.29381
differential_evolution step 10: f(x)= 7.29381
differential_evolution step 11: f(x)= 7.29381
differential_evolution step 12: f(x)= 7.29381
differential_evolution step 13: f(x)= 7.29381
differential_evolution step 14: f(x)= 7.29381
differential_evolution step 15: f(x)= 7.29381
differential_evolution step 16: f(x)= 7.29381
differential_evolution step 17: f(x)= 7.29381
differential_evolution step 18: f(x)= 7.29381
differential_evolution step 19: f(x)= 7.29381
differential_evolution step 20: f(x)= 7.29381
differential_evolution step 21: f(x)= 7.29381
differential_evolution step 22: f(x)= 7.29381
differential_evolution step 23: f(x)= 7.29381
differential_evolution step 24: f(x)= 7.2

differential_evolution step 4: f(x)= 7.01106
differential_evolution step 5: f(x)= 7.01106
differential_evolution step 6: f(x)= 7.01106
differential_evolution step 7: f(x)= 7.01106
differential_evolution step 8: f(x)= 7.01106
differential_evolution step 9: f(x)= 7.01106
differential_evolution step 10: f(x)= 6.93007
differential_evolution step 11: f(x)= 6.93007
differential_evolution step 12: f(x)= 6.93007
differential_evolution step 13: f(x)= 6.93007
differential_evolution step 14: f(x)= 6.93007
differential_evolution step 15: f(x)= 6.93007
differential_evolution step 16: f(x)= 6.93007
differential_evolution step 17: f(x)= 6.93007
differential_evolution step 18: f(x)= 6.93007
differential_evolution step 19: f(x)= 6.93007
differential_evolution step 20: f(x)= 6.93007
differential_evolution step 21: f(x)= 6.93007
differential_evolution step 22: f(x)= 6.93007
differential_evolution step 23: f(x)= 6.93007
differential_evolution step 24: f(x)= 6.93007
differential_evolution step 25: f(x)= 6.

differential_evolution step 11: f(x)= 6.86376
differential_evolution step 12: f(x)= 6.86376
differential_evolution step 13: f(x)= 6.86376
differential_evolution step 14: f(x)= 6.86376
differential_evolution step 15: f(x)= 6.86376
differential_evolution step 16: f(x)= 6.86376
differential_evolution step 17: f(x)= 6.86376
differential_evolution step 18: f(x)= 6.86376
differential_evolution step 19: f(x)= 6.86376
differential_evolution step 20: f(x)= 6.86376
differential_evolution step 21: f(x)= 6.86376
differential_evolution step 22: f(x)= 6.86376
differential_evolution step 23: f(x)= 6.86376
differential_evolution step 24: f(x)= 6.86376
differential_evolution step 25: f(x)= 6.86376
differential_evolution step 26: f(x)= 6.86376
differential_evolution step 27: f(x)= 6.86376
differential_evolution step 28: f(x)= 6.86376
differential_evolution step 29: f(x)= 6.86376
differential_evolution step 30: f(x)= 6.86376
differential_evolution step 31: f(x)= 6.83739
differential_evolution step 32: f(

In [35]:
result

     fun: 6.53615095982203
     jac: array([ 0.00254365,  0.00448885,  0.01040998, -0.00292495,  0.02470779,
       -0.00434008,  0.00577822, -0.00553309,  0.00559455,  0.01923643,
        0.01338831, -0.00623013,  0.00419673, -0.00944071, -0.08234533,
        0.01047091,  0.02148415,  0.01330305])
 message: 'Optimization terminated successfully.'
     nit: 18
 success: True
       x: array([-156.09747468,  -17.27594582,   49.75982598, -150.2155858 ,
         48.30684463,    0.76582708, -130.27795607,    3.36513068,
         53.36842297,   42.95301131,  -57.77981561,  -55.30469806,
        -65.14724915,   79.40574001,  200.        ,  -89.08757415,
       -200.        , -164.68229866])